In [1]:
from typing import List
import hydra
import hydra.experimental
import torch
import numpy as np
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from omegaconf import DictConfig

from model import Seq2Seq
from torchfly.text.decode import TransformerDecoder

In [2]:
config = {
    "max_steps": 100,
    "num_return_sequences": 1,
    "num_beams": 4,
    "do_sample": True,
    "top_k": -1,
    "top_p": 0.9,
    "temperature": 1.0,
    "repetition_penalty": 1.0,
    "length_penalty": 1.0,
    "bos_token_id": None,
    "eos_token_ids": [13],
    "output_log_probs": True,
}

config = DictConfig(config)

In [3]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [4]:
class SimpleGenerativeModel(TransformerDecoder):
    def __init__(self, config):
        super().__init__(config)
        self.model = GPT2LMHeadModel.from_pretrained("gpt2")
        self.register_generator(self.model)
        self.register_tokenizer(tokenizer)

In [5]:
model = SimpleGenerativeModel(config)

In [6]:
sentence = tokenizer.encode("A dog can eat an")
input_ids = torch.LongTensor([sentence]).cuda()
model.model.cuda()
pass

Using pad_token, but it is not set yet.


In [7]:
model_inputs = model.prepare_model_inputs_for_generation(input_ids)

In [8]:
model.batch_size = 1

In [9]:
results = model._generate_beam_search(model_inputs)

> /home/wuqy1203/Desktop/Projects/TorchFly_dev/torchfly/text/decode/transformer_decoder.py(442)_generate_beam_search()
-> generated_hyps[batch_idx].add(


(Pdb)  beam_log_prob_scores_1d


tensor([[-4.1104],
        [-4.2550],
        [-5.0541],
        [-6.8365]], device='cuda:0', grad_fn=<ViewBackward>)


(Pdb)  beam_log_prob_sequence.sum(-1)


tensor(-4.3603, grad_fn=<SumBackward1>)


...

tensor(25995)


(Pdb)  beam_log_prob_sequence.sum(-1)


tensor(-41.4416, grad_fn=<SumBackward1>)


(Pdb)  q


BdbQuit: 

In [ ]:
[item.sum() for item in results['log_probs']]

In [ ]:
results['log_probs']

In [ ]:
results['beam_scores']

In [ ]:
results['tokens']

In [ ]:
tokenizer.decode(results['tokens'][3].tolist())

In [ ]:
np.exp(np.ln(0.4) / np.log(2))

In [ ]:
np.exp(np.log(0.4) - torch.log(2))

In [ ]:
import math